In [61]:
model_checkpoint = "microsoft/swin-tiny-patch4-window7-224" # pre-trained model from which to fine-tune
batch_size = 32 # batch size for training and evaluation

In [62]:
!pip install -q transformers

In [63]:
from huggingface_hub import notebook_login

notebook_login()

In [64]:
# %%capture
# !sudo apt -qq install git-lfs
# !git config --global credential.helper store

In [48]:
# Read the images and labels of gemstones from training data
def extract_labels(name):
    parts = name.split('/')
    train = parts[1]
    if 'Fake' in parts[2]:
        label = 'fake'
    else:
        label = 'real'
    return train, label
    

def read_images():
    Images = []
    for root, dirs, files in os.walk('datasets'):
        f = os.path.basename(root)
        for file in files:
            try:
                image = cv2.imread(root+'/'+file)
                image = np.array(image)
                train, label = extract_labels(root)
                Images.append([train, image, label])
            except Exception as e:
                print(e)
    return Images  

In [51]:
images = read_images()
df = pd.DataFrame(images, columns = ['train', 'image', 'label'])

In [58]:
df.head()

,train,image,label
0,test,"[[[152, 156, 157], [152, 156, 157], [152, 156,...",fake
1,test,"[[[152, 156, 157], [152, 156, 157], [152, 156,...",fake
2,test,"[[[152, 156, 157], [152, 156, 157], [152, 156,...",fake
3,test,"[[[152, 156, 157], [152, 156, 157], [152, 156,...",fake
4,test,"[[[152, 156, 157], [152, 156, 157], [152, 156,...",fake


In [65]:
# Preprocessing

from transformers import AutoImageProcessor

image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)
image_processor 

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [69]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch